In [10]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel #tensorflow GPT2 LM
import numpy as np
import torch
from tqdm import tqdm  # 진행률 표시를 위한 tqdm 라이브러리

mirrored_strategy = tf.distribute.MirroredStrategy()
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.10.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'lm_head.weight', 'transformer.h.3.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.9.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

In [11]:
filename = 'prompt.txt'
f = open(filename, 'r')
lines = f.readlines()
print(lines)
print('-' * 10)

['아침 일과 시작하기\n', '책 읽기의 즐거움\n', '가족과 함께 하는 저녁 식사\n', '친구와의 영화 관람\n', '주말 동안의 여행 계획\n', '정원 가꾸기의 효과\n', '요리를 통한 스트레스 해소\n', '자전거 타기의 장점\n', '집안 정리 정돈 팁\n', '개인 운동 루틴 유지하기\n', '취미 활동의 중요성\n', '새로운 언어 배우기\n', '음악 연주와 그 영향\n', '사진 촬영을 통한 추억 만들기\n', '지역 사회 봉사의 보람\n', '패션과 개인 스타일 표현\n', '휴일을 활용한 가족 모임\n', '건강한 식생활을 위한 조리법\n', '애완동물과 보내는 시간\n', '독서 클럽에서의 토론\n', '도서관 방문의 장점\n', '집에서의 필름 감상\n', '커피 한 잔과의 여유\n', '일기 쓰기의 치유 효과\n', '홈 베이킹의 즐거움\n', '직접 만든 공예품\n', '새로운 요리법 시도하기\n', '지역 공연 관람\n', '자연 속에서의 하이킹\n', '가구 리폼 프로젝트\n', '아이와 함께하는 공원 나들이\n', '새로운 스포츠 도전\n', '개인 기록의 개선\n', '집에서의 요가 세션\n', '반려동물과의 산책\n', '가족 앨범 만들기\n', '매일의 명상 시간\n', '가벼운 아침 조깅\n', '주말 브런치 준비\n', '일상의 소소한 행복 찾기\n', '스마트폰 없는 하루\n', '책상 정리의 중요성\n', '옷장 정리하기\n', '새벽 시간의 고요함\n', '오후의 티타임\n', '자기 전 명상\n', '집안 식물 가꾸기\n', '근처 박물관 탐방\n', '집에서의 칵테일 파티\n', '커뮤니티 클래스 참여\n', '셀프 케어 루틴\n', '장거리 걷기의 이점\n', '텃밭 가꾸기\n', '개인적인 스타일 개발\n', '책 추천 공유\n', '로컬 마켓 쇼핑\n', '장기 게임의 즐거움\n', '가족과의 보드게임 밤\n', '디지털 디톡스\n', '가까운 도시 탐험\n', '수제 비누 만들기\n', '로컬 아

In [15]:

with open('perplexity_g=0.5_d=1.0.txt', 'w') as f:  #file내용 clear
    f.write('')

for line in lines:
    line = line.strip()
    print(f"prompt : {line}")

    input_text = line
    input_ids = tokenizer.encode(input_text, return_tensors='tf')
    #term = 3

    last_index = input_ids.shape[-1]
    nlls = []
            
    with mirrored_strategy.scope():
        for _ in tqdm(range(100)):  # 최대 100번 반복
            if input_ids.shape[-1] >= 100:
                break  # input_ids의 길이가 100 이상이면 반복 중단
            
            #print("================================")
            #print(f"문장의 토큰 개수 : {input_ids.shape[-1]}")
            #set 나눠서 green set 로짓을 촉진하기
            gamma = 0.5
            delta = 1.0
            Logits = model(input_ids).logits
            #print(f"original Logits : {Logits}")
            
            #seed 생성
            seed = input_ids[-1] % Logits.shape[-1]
            #print(f"seed : {seed[-1]}")
            np.random.seed(seed[-1])

            #random green list 설정
            indices = np.arange(Logits.shape[-1]) #0~51999
            green_list_size = int(Logits.shape[-1] * gamma)  #25600
            green_list = np.random.choice(indices, green_list_size, replace=False)
            #print(f"green list : {green_list} {green_list.shape}") #25600개의 green list

            #--여기부터 probability 구하기
            # green logit과 red logit의 값을 담을 리스트 생성
            green_logits = tf.zeros(green_list_size)
            red_logits = tf.zeros(Logits.shape[-1] - green_list_size)

            #그린 리스트에 속한 idx 값의 Logit에는 델타 추가!!!!!!
            Logits_num = np.zeros((0,last_index-1,Logits.shape[-1]))
            Logits_num = Logits.numpy()
            for i in green_list :
                Logits_num[0,last_index-1,i] = Logits_num[0,last_index-1,i]+delta
            Logits = tf.convert_to_tensor(Logits_num, dtype=tf.float32)
            #print(f"updated Logits : {Logits}")
            
            # softmax 계산을 위한 분모
            denominator = tf.reduce_sum(tf.exp(Logits), axis=-1)  
            #print(f"denominator : {denominator}") #25600개의 green list

            # Logits 텐서에 softmax 적용하여 확률 계산 (병렬 계산), softmax_tensor = probability
            softmax_tensor = Logits.numpy()
            softmax_tensor[0, last_index-1, :] = np.exp(softmax_tensor[0, last_index-1, :])
            softmax_tensor[0, last_index-1, :] /= denominator[0,last_index-1]
            #print(softmax_tensor.shape)
            #print(f"Probabilities : {softmax_tensor}")

            # 데이터 타입 확인 및 조정
            add_token_id = tf.argmax(softmax_tensor[0][last_index-1])
            #print(f"updated Logits Max value : {add_token_id}")
            
            # Calculate perplexity
            # 참고 https://huggingface.co/docs/transformers/main/ko/perplexity
            selected_probability = softmax_tensor[0, last_index-1, add_token_id]
            nlls.append(np.log(selected_probability))
            
            #print(softmax_tensor[0, last_index-1, add_token_id])

            # 텐서 연결
            add_token_id = tf.reshape(add_token_id, [1,1])
            add_token_id = tf.cast(add_token_id, input_ids.dtype)
            input_ids = tf.concat([input_ids, add_token_id], axis=-1)
            next_token = tokenizer.decode(input_ids.numpy()[0,:])
            #print(f"next token : {next_token}")
            
            last_index = input_ids.shape[-1]
            #term = 3
        final_token = tokenizer.decode(input_ids.numpy()[0,:])
        print(final_token)

        # Calculate perplexity
        perplexity = tf.exp(-tf.reduce_mean(nlls))
        print("Perplexity: ", perplexity.numpy())
        
        perplexity_str = str(perplexity.numpy())  # perplexity를 문자열로 변환
        with open('perplexity_g=0.5_d=1.0.txt', 'a') as f:  # with 문을 사용하여 파일을 열면 자동으로 닫힙니다.
            f.write(perplexity_str + '\n')

prompt : 아침 일과 시작하기


 96%|█████████▌| 96/100 [00:17<00:00,  5.60it/s]


아침 일과 시작하기 전에 미리 준비해두면 좋겠습니다.
오늘은 전국이 대체로 맑겠습니다.
하지만 제주도는 구름이 많이 끼겠고요.
강원 영동 지역은 밤 한때 비가 조금 내리겠습니다.
내일 아침 기온 오늘과 비슷하겠습니다.
서울이 4도, 대구 6도로 출발하겠습니다.
낮 기온은 서울이 13도, 대구와 광주 16도로 오늘과 비슷하겠습니다.
바다의 물결은 동해 먼바다에서 최고 2.5미터까지 높게 일겠습니다.
내일 아침 기온 오늘과 비슷하겠습니다.

Perplexity:  2.4171393
prompt : 책 읽기의 즐거움


 95%|█████████▌| 95/100 [00:16<00:00,  5.64it/s]


책 읽기의 즐거움도 함께 느낄 수 있다.
이번 전시는 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 진행된다.
전시에서는 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는
Perplexity:  2.0883842
prompt : 가족과 함께 하는 저녁 식사


...

지역사회와의 소통 강화 및 지역경제 활성화를 위해 노력하겠다"고 말했다.
한편 이날 행사에는 박성효 대전시장과 김관용 경북도지사, 김관용 경북도지사, 김관용 경북도지사, 김관용 경북도지사, 김관용 경북도지사, 김관용 경북도지사, 김관용 경북도지사, 김관용 경북도지사, 김관용 경북도지사, 김관용 경북도지사, 김관용 경북도지사,
Perplexity:  1.7284392
prompt : 가족과의 소중한 대화


 96%|█████████▌| 96/100 [00:17<00:00,  5.60it/s]


가족과의 소중한 대화에서 '진심 어린 사과'를 해야 한다"고 말했다.
이어 "국민이 납득할 수 있는 사과를 해야 한다"며 "국민이 납득할 수 있는 사과를 해야 한다"고 덧붙였다.
이날 청와대 수석비서관회의에서는 박 대통령의 대국민 사과를 두고도 논란이 이어졌다.
박 대통령은 "국민이 납득할 수 있는 사과를 해야 한다"며 "국민이 납득할 수 있는 사과를 해야 한다"고 말했다.
이에 대해 이 수석비서
Perplexity:  2.805211
prompt : 개인 발전을 위한 목표 설정


 95%|█████████▌| 95/100 [00:17<00:00,  5.58it/s]

개인 발전을 위한 목표 설정과 실행, 그리고 그 과정에서 발생하는 다양한 문제들을 해결하는 데 있어 많은 노력을 기울이고 있다.
이러한 노력의 결과로 인해, 우리는 많은 부분에서 많은 부분에서 많은 부분에서 많은 부분에서 많은 부분에서 많은 부분에서 많은 부분에서 많은 부분에서 많은 부분에서 많은 부분에서 많은 부분에서 많은 부분에서 많은 부분에서 많은 부분에서 많은 부분에서 많은 부분들이 많은 부분들이 있다.
이러한 부분들은 우리가 생각하는 것처럼 우리가 생각하는 것처럼 우리가 생각하는 것처럼 우리가 생각하는 것처럼 우리가 생각하는 것처럼 우리가 생각하는 것처럼 우리가 생각하는 것처럼 우리가 생각하는 것처럼 우리가 생각하는 것처럼 우리가 생각하는
Perplexity:  4.615148


In [18]:
# 첫 번째 파일에서 숫자 읽어오기
with open('perplexity.txt', 'r') as f:
    list1 = [float(line.strip()) for line in f]

# 두 번째 파일에서 숫자 읽어오기
with open('perplexity_g=0.5_d=1.0.txt', 'r') as f:
    list2 = [float(line.strip()) for line in f]

# 두 번째 리스트에서 더 큰 값을 가진 경우를 세기 위한 카운터 초기화
count_second_list_bigger = 0

# 두 리스트를 순회하면서 두 번째 리스트가 더 큰 경우를 세어줌
for val1, val2 in zip(list1, list2):
    if val2 > val1:  # 두 번째 리스트의 값이 더 큰 경우
        count_second_list_bigger += 1

# 결과 출력
print("두 번째 리스트에서 더 큰 값을 가진 경우의 수:", count_second_list_bigger)


두 번째 리스트에서 더 큰 값을 가진 경우의 수: 53


: 

In [5]:
#delta = 10.0, gamma = 0.5 해서 비교해보기

tf.Tensor(888.30756, shape=(), dtype=float32)
